# U-Net with Bayesian Optimisation

## WORK IN PROGRESS !! Not finished

In [ ]:
import tensorflow as tf
import os
import random
import numpy as np
 
from tqdm import tqdm 

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.measure import label, regionprops
from skimage.util import random_noise

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


from tensorflow import keras
from datetime import datetime

from keras import layers, metrics


from tensorboard.plugins.hparams import api as hp


from sklearn.metrics import jaccard_score

from bayes_opt import BayesianOptimization

In [ ]:
date = "24_04_22"

In [ ]:
def display(display_list):
  plt.figure(figsize=(8, 8))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    if i == 1:
      plt.imshow(display_list[i], cmap='gray',  interpolation='nearest')
    elif i == 2:
      plt.imshow(display_list[i], cmap='jet',  interpolation='nearest')
    else:
      plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
#image dimensions, seems to work with non-square inputs
IMAGE_CHANNELS = 3

IMAGE_HEIGHT =  192
IMAGE_WIDTH = 64

seed = 4
np.random.seed = seed
random.seed(seed)
tf.random.set_seed(seed)

DATA_TRAIN = "./datasets/KolektorSDD2/train/"
DATA_TEST = "./datasets/KolektorSDD2/test/"



train_ids = next(os.walk(os.path.join(DATA_TRAIN, "images/")))[2]
test_ids = next(os.walk(os.path.join(DATA_TEST, "images/")))[2]

damaged = [] # prepare for data augmentation
damaged_mask = []


X_train = np.zeros((len(train_ids), IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS), dtype=np.float16)
y_train = np.zeros((len(train_ids), IMAGE_HEIGHT, IMAGE_WIDTH), dtype=np.float16)

print('Resizing training images and masks')

for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):   
    path = DATA_TRAIN 

    img = imread(path + 'images/' + id_)[:,:,:IMAGE_CHANNELS]  
    img = resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
    img /= 255.0
    X_train[n] = img  #Fill empty X_train with values from img

    mask = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=bool)
    mask_file = os.path.join(path + 'masks/' + id_[:5] + "_GT.png")
    mask = imread(mask_file)[:,:]

    mask = resize(mask, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
    mask /= 255.0  
    mask = np.where(mask > 0.5, 1.0, 0.0) 
    y_train[n] = mask 

    if np.count_nonzero(mask) != 0:
        damaged.append(img)
        damaged_mask.append(mask)
    
# test images
test_images = np.zeros((len(test_ids), IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS), dtype=np.float16)
test_masks = np.zeros((len(test_ids), IMAGE_HEIGHT, IMAGE_WIDTH), dtype=np.float16)

sizes_test = []
print('Resizing test images') 
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = DATA_TEST
    img = imread(path + '/images/' + id_ )[:,:,:IMAGE_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
    img /= 255.0
    test_images[n] = img

    mask = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=bool)
    mask_file = os.path.join(path + 'masks/' + id_[:5] + "_GT.png")
    mask = imread(mask_file)[:,:]

    mask = resize(mask, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
    mask /= 255.0     
    mask = np.where(mask > 0.5, 1.0, 0.0)   
    test_masks[n] = mask 

## Data augmentation - rotate and flip images
vertical_train = np.flip(damaged, axis=0)
vertical_test = np.flip(damaged_mask, axis=0)

horizontal_train = np.flip(damaged, axis=1)
horizontal_test = np.flip(damaged_mask, axis=1)

rotating_train = np.rot90(damaged, k=2)
rotating_test = np.rot90(damaged_mask, k=2)

vert_rot_train = np.rot90(vertical_train, k=2)
vert_rot_test = np.rot90(vertical_test, k=2)

hor_rot_train = np.rot90(horizontal_train, k=2)
hor_rot_test = np.rot90(horizontal_test, k=2)

# Done with rotation
Boxes = []
check = []
for img in damaged_mask:
    labels = label(img)
    regions = regionprops(labels)
    if len(regions) == 1:
        check.append(1)
        for props in regions:
            min_x, min_y, max_x, max_y = props.bbox
            Boxes.append((min_x, min_y, max_x, max_y))
    else:
        check.append(0)

# Throw out images, which have more than one damage
onedamage = [damaged[i] for i in range(len(damaged)) if check[i] == 1]
onedamage_mask = [damaged_mask[i] for i in range(len(damaged_mask)) if check[i] == 1]

def crop_image(image, bbox):
    # Crop the image using NumPy array slicing
    cropped_image = image[bbox[0]:bbox[2], bbox[1]:bbox[3]]
    return cropped_image

def overlay_image(background, background_mask, overlay, overlay_mask):
    # Generate random position for overlay image
    if overlay.shape[1] < background.shape[1]:
        x_offset = np.random.randint(0, background.shape[1] - overlay.shape[1])
        y_offset = np.random.randint(0, background.shape[0] - overlay.shape[0])
    
        # Overlay the image
        background[y_offset:y_offset + overlay.shape[0], x_offset:x_offset + overlay.shape[1]] = overlay
        background_mask[y_offset:y_offset + overlay_mask.shape[0], x_offset:x_offset + overlay_mask.shape[1]] = overlay_mask
        return background, background_mask
    else:
        return background, background_mask

generated_img = np.empty((len(onedamage),IMAGE_HEIGHT,IMAGE_WIDTH,3))
generated_mask = np.empty((len(onedamage), IMAGE_HEIGHT,IMAGE_WIDTH))
overlayed_indices = []
for i, image in enumerate(onedamage):
    # Get the bounding box for the current image
    bbox = Boxes[i]
    mask = onedamage_mask[i]
    # Crop the image
    cropped_image = crop_image(image, bbox)
    cropped_mask = crop_image(mask, bbox)
    # Pick a random overlay image from the list
    overlay_image_index = np.random.choice([idx for idx in range(len(X_train)) if idx not in overlayed_indices])
    overlay = X_train[overlay_image_index]
    overlay_mask = y_train[overlay_image_index]

    # Overlay the cropped image onto the random overlay image
    new_image, new_mask = overlay_image(overlay, overlay_mask, cropped_image, cropped_mask)
    generated_img[i] = new_image
    generated_mask[i] = new_mask
    overlayed_indices.append(overlay_image_index)

noised = np.empty_like(damaged)
noised_mask = damaged_mask
i = 0
for img in damaged:
    noise = random_noise(img, mode='gaussian', rng=seed, clip=True)
    noised[i] = noise
    i = i+1

noised_vert = np.empty_like(damaged)
noised_mask_vert = vertical_test
i = 0
for img in vertical_train:
    noise = random_noise(img, mode='gaussian', rng=seed, clip=True)
    noised_vert[i] = noise
    i = i+1

noised_horr = np.empty_like(damaged)
noised_mask_horr = horizontal_test
i = 0
for img in horizontal_train:
    noise = random_noise(img, mode='gaussian', rng=seed, clip=True)
    noised_horr[i] = noise
    i = i+1

X_train = np.concatenate((vertical_train, horizontal_train, rotating_train, vert_rot_train, hor_rot_train, X_train))
y_train = np.concatenate((vertical_test, horizontal_test, rotating_test, vert_rot_test, hor_rot_test, y_train))

print('Dataset is ready')

In [ ]:
def train_test_model(learning_rate, channels, batch_size):
    # (X_train, y_train) = dataset
    global X_train, y_train, test_images, test_masks
    non_zero = np.count_nonzero(y_train)
    print(non_zero)
    # print(non_zero/(IMAGE_HEIGHT*IMAGE_WIDTH*len(y_train))*100)

    unique, counts = np.unique(y_train, return_counts=True)

    print(dict(zip(unique, counts)))
    print("Percentage of faulty images in train data:", counts[1]/(counts[0]+counts[1])*100, " %")
    neg = counts[0]
    pos = counts[1]
   
    initial_bias = np.log([pos/neg])
    output_bias = tf.keras.initializers.Constant(initial_bias)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = seed)
    

    #Building U-net model
    #Downward stream
    inputs = tf.keras.layers.Input((IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS))
    conv_11 = layers.Conv2D(16*channels,kernel_size=(3,3), activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(inputs)
    conv_12 = layers.Conv2D(16*channels,kernel_size=(3,3), activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_11)#TODO: Understand parameters

    max_pool_1 = layers.MaxPool2D((2,2))(conv_12)
    conv_21 = layers.Conv2D(32*channels,(3,3),activation = 'relu',padding= 'same',kernel_initializer = 'he_normal')(max_pool_1)
    conv_22 = layers.Conv2D(32*channels,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_21)

    max_pool_2 = layers.MaxPool2D((2,2))(conv_22)
    conv_31 = layers.Conv2D(64*channels,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_2)
    conv_32 = layers.Conv2D(64*channels,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_31)

    max_pool_3 = layers.MaxPool2D((2,2))(conv_32)
    conv_41 = layers.Conv2D(128*channels,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_3)
    conv_42 = layers.Conv2D(128*channels,(3,3),activation = 'relu', padding= 'same')(conv_41)

    max_pool_4 = layers.MaxPool2D((2,2))(conv_42)
    conv_51 = layers.Conv2D(256*channels,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_4)
    conv_52 = layers.Conv2D(256*channels,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_51)

    #Upward stream
    upconv_1 = layers.Conv2DTranspose(128*channels,(2,2), strides=(2,2))(conv_52)
    upconv_1_conc = layers.concatenate([upconv_1,conv_42])
    conv_61 = layers.Conv2D(128*channels,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_1_conc)
    conv_62 = layers.Conv2D(128*channels,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_61)

    upconv_2 = layers.Conv2DTranspose(64*channels, (2,2), strides = (2,2))(conv_62)
    upconv_2_conc = layers.concatenate([upconv_2, conv_32])
    conv_71 = layers.Conv2D(64*channels,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_2_conc)
    conv_72 = layers.Conv2D(64*channels,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_71)

    upconv_3 = layers.Conv2DTranspose(32*channels,(2,2), strides=(2,2))(conv_72)
    upconv_3_conc = layers.concatenate([upconv_3,conv_22])
    conv_81 = layers.Conv2D(32*channels,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_3_conc)
    conv_82 = layers.Conv2D(32*channels,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_81)

    upconv_4 = layers.Conv2DTranspose(16*channels,(2,2), strides=(2,2))(conv_82)
    upconv_4_conc = layers.concatenate([upconv_4,conv_12])
    conv_91 = layers.Conv2D(16*channels,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_4_conc)
    conv_92 = layers.Conv2D(16*channels,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_91)
    outputs = layers.Conv2D(1,(1,1), activation = 'sigmoid', padding = 'same',kernel_initializer = 'he_normal', bias_initializer=output_bias)(conv_92)#TODO: Check function here

    model = tf.keras.Model(inputs = [inputs], outputs = [outputs])

    from keras.optimizers import Adam, SGD
    optimizer = Adam(learning_rate = learning_rate)
    # optimizerSGD = SGD(learning_rate = learning_rate)

    #Compiling model
    model.compile(optimizer=optimizer , loss='binary_crossentropy', metrics=[metrics.BinaryIoU(threshold=0.5)]) #TODO: Parameters check #metrics.BinaryIoU(), 'accuracy'
    # model.compile(optimizer=optimizerSGD, loss='mse', metrics=[metrics.BinaryIoU(
                                                            # target_class_ids=[0],
                                                            # threshold=0.5)]) #TODO: Parameters check #metrics.BinaryIoU(), 'accuracy'
    model.summary()


    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, restore_best_weights=True)

    epochs = 100  
    batch_size = batch_size

    # run_name = "/run-%d" % session_num
    # logdir = f"logs/hparam_tuning_{date}/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S") + run_name
    # tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

    history = model.fit(X_train, 
                        y_train, 
                        epochs = epochs, 
                        validation_data = (X_val, y_val), 
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        callbacks=[early_stopping],
                        batch_size = batch_size,) 

    # Y_pred = model.predict(test_images)
    
    # Model Loss
    plot_folder = f"./plots/{date}"
    os.makedirs(plot_folder, exist_ok=True)

    # plt.figure()
    # plt.plot(history.history["loss"],label = "Train Loss", color = "black")
    # plt.plot(history.history["val_loss"],label = "Validation Loss", color = "darkred", marker = "+", linestyle="dashed", markeredgecolor = "purple", markeredgewidth = 2)
    # plt.title(f"Model Loss - session {session_num}", color = "darkred", size = 13)
    # plt.legend()
    # plt.savefig(f"plots/{date}/loss_session_{session_num}.png")
    # plt.close()
    # plt.show()

    
    # model_folder = f"./trainedModels/hptuning_{date}"
    # os.makedirs(model_folder, exist_ok=True)

    # model.save(f"trainedModels/hptuning_{date}/hptuning_session{session_num}.h5")

    # Model Jaccard score
    threshold_list = np.linspace(0.0, 0.4, 30)
    jaccard_scores = []
    Y_pred = model.predict(test_images)
    true_masks_flat = test_masks.reshape(test_masks.shape[0], -1)
    for threshold in threshold_list:
        y_pred_binary = (Y_pred >= threshold).astype(int)
        pred_masks_binary_flat = y_pred_binary.reshape(y_pred_binary.shape[0], -1)
        jaccard_scores.append(jaccard_score(true_masks_flat, pred_masks_binary_flat, average="micro"))
    print("Best jacard score: ", max(jaccard_scores))
    
    accuracy = max(jaccard_scores)


    return accuracy

In [ ]:
# Define parameter space
pbounds = {'learning_rate': (1e-5, 1e-2),
           'channels': (1, 2),
           'batch_size': (16, 128),
        #    'num_filters': (16, 128),
        #    'kernel_size': (3, 5),
        #    'pool_size': (2, 4)
        }

# Bayesian Optimization
optimizer = BayesianOptimization(f=train_test_model, pbounds=pbounds, random_state=4, verbose=2)
optimizer.maximize(init_points=5, n_iter=5)

# Print best hyperparameters
print("Best hyperparameters:", optimizer.max)